In [55]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import json
import time
import pymongo
from pymongo import MongoClient
import re
import numpy as np
import pandas as pd

We'll first do the following steps to scrap from ebay.com. We'll be using a function to do the same.

a) Using selenium driver to go to ebay.com 

b) Search for "Cell Phones" in the website using the same driver

c) Then select all the below filters,
-  Network: Unlocked
- Brand: LG
- Screen Size: 6 in or More
- Storage Capacity: 128 GB
- Lock Status: Factory Unlocked
- Condition: Used

d) Store the resulting html page locally

In [13]:
def scrap_ebay():
    '''
    Function to srap ebay and store the html file locally.
    '''

    #Create a driver object for the chrome 
    driver = webdriver.Chrome()
    
    #We'll wait for 5 sec before sending each request
    
    #Go to ebay.com
    time.sleep(5)
    driver.get('https://ebay.com/')
    
    #search for "Cell Phones"
    time.sleep(5)
    input = driver.find_element(By.CSS_SELECTOR,'input[placeholder="Search for anything"]')
    input.send_keys('Cell Phones \n')
    
    #Selecting and clicking all the required filtering options 
    
    time.sleep(5)
    network = driver.find_element(By.XPATH,'//span[text()="Unlocked"]')
    network.click()
    
    time.sleep(5)
    brand = driver.find_element(By.XPATH,'//span[text()="LG"]')
    brand.click()
    
    time.sleep(5)
    screen_size = driver.find_element(By.XPATH,'//span[text()="6 in or More"]')
    screen_size.click()
    
    time.sleep(5)
    capacity = driver.find_element(By.XPATH,'//span[text()="128 GB"]')
    capacity.click()
    
    time.sleep(5)
    lock_status = driver.find_element(By.XPATH,'//span[text()="Factory Unlocked"]')
    lock_status.click()
    
    time.sleep(5)
    condition = driver.find_element(By.XPATH,'//span[text()="Used"]')
    condition.click()
    
    #Saving the HTML of the resulting page 
    html_page = driver.page_source
    
    with open('./unlocked-lg-128gb-used.html', 'w', encoding='utf-8') as file:
                file.write(str(html_page))


scrap_ebay()

e) We'll now read the save html file and parse it using Beautiful Soup

In [4]:
#Read the html file 
with open('./unlocked-lg-128gb-used.html', 'r', encoding='utf-8') as file:
    html = file.read()

soup = BeautifulSoup(html,'html')

f) Let us now extract and print the following information for each listed item,

- Title
- Seller
- Seller rating (percentage)
- Price
- Shipping information (if available)
- Return information (if available)
- Number of bids (if applicable) 

In [58]:
#Collection of all divisions of 'li' with this class 
collection = soup.find_all('li',class_="s-item s-item__pl-on-bottom")

#We know there are only 60 listings per page, and we have only stored the first page, we'll be running the loop from 1:60 as the 0th element
#is not a listing under this collection 

#For storing all the output info dicts in an array for dataframe purpose at the end
data=[] 

for listing in collection[1:60]:
    #Title
    title= listing.find('span',role='heading').text

    #Seller Details 
    seller_details = listing.find('span',class_="s-item__seller-info-text").text

    #Seller 
    seller = re.findall(r'(.+) \(.+',seller_details)[0]

    #Seller rating percentage 
    seller_rating_pct = re.sub(r'(.+) \(.+\) (.+)%','\\2',seller_details) 

    #Shipping information 
    shipping = listing.find('span',class_="s-item__shipping s-item__logisticsCost").text if listing.find('span',class_="s-item__shipping s-item__logisticsCost") is not None else "Not Available"

    #Return information 
    return_info = listing.find('span',class_="s-item__free-returns s-item__freeReturnsNoFee").text if listing.find('span',class_="s-item__free-returns s-item__freeReturnsNoFee") is not None else "Not Available"

    #Number of bids 
    bids = re.findall(r'(\d+) bids*',listing.find('span',class_="s-item__bids s-item__bidCount").text)[0] if listing.find('span',class_="s-item__bids s-item__bidCount") is not None else np.NaN

    listing_info = {"title":title, "seller":seller, "seller_rating_pct":seller_rating_pct, "shipping_info":shipping, "return_info":return_info, "no_of_bids":bids}

    data.append(listing_info)
    print(listing_info,"\n")

{'title': 'LG Velvet 5G 128GB Grey LM-G900TM (T-Mobile Unlocked) Reduced Price zW7656', 'seller': 'soonersoft', 'seller_rating_pct': '98.8', 'shipping_info': 'Free shipping', 'return_info': 'Free returns', 'no_of_bids': nan} 

{'title': 'LG Velvet 5G - LM-G900VMP 128GB Gray Verizon + GSM Unlocked LTE Smartphone GREAT', 'seller': 'discountphonedepot', 'seller_rating_pct': '97.5', 'shipping_info': 'Free shipping', 'return_info': 'Free returns', 'no_of_bids': nan} 

{'title': 'LG K61 LG-Q630UM 128GB Unlocked White Android Smartphone', 'seller': 'monkeytote', 'seller_rating_pct': '98.5', 'shipping_info': 'Free shipping', 'return_info': 'Not Available', 'no_of_bids': nan} 

{'title': 'LG Velvet 5G 128GB Gray LM-G900TM (Unlocked) - GSM World Phone - DF8149', 'seller': 'soonersoft', 'seller_rating_pct': '98.8', 'shipping_info': 'Free shipping', 'return_info': 'Free returns', 'no_of_bids': nan} 

{'title': 'New ListingLG Velvet 5G LM-G900UM G900 (Unlocked) 128GB 6.8" - Good Condition', 'seller

In [61]:
#The listing information with a dataframe, which can be used for further analysis

df_listing = pd.DataFrame(data)
df_listing

#Additional: The shiffing_info can later be convered into shipping_price if required for future analysis i.e 0 for free shipping in dollars 
#while extracting the information of shipping charges through regex from other available shipping price values.

,title,seller,seller_rating_pct,shipping_info,return_info,no_of_bids
0,LG Velvet 5G 128GB Grey LM-G900TM (T-Mobile Un...,soonersoft,98.8,Free shipping,Free returns,NaN
1,LG Velvet 5G - LM-G900VMP 128GB Gray Verizon +...,discountphonedepot,97.5,Free shipping,Free returns,NaN
2,LG K61 LG-Q630UM 128GB Unlocked White Android ...,monkeytote,98.5,Free shipping,Not Available,NaN
3,LG Velvet 5G 128GB Gray LM-G900TM (Unlocked) -...,soonersoft,98.8,Free shipping,Free returns,NaN
4,New ListingLG Velvet 5G LM-G900UM G900 (Unlock...,wikiwoo,98.9,Free shipping,Free returns,NaN
5,"LG Velvet 5G G900UM 128GB Aurora Silver 6.8"" S...",cucoins88,99.6,+$6.15 shipping,Free returns,NaN
6,LG G8 Thinq - LM-G820 - 128GB - Black (T-Mobil...,katiesriches,97.1,+$6.15 shipping,Not Available,0
7,LG G8 ThinQ 4G 5G LM-G820QM 128GB Smartphone -...,electronicsradar,96.9,Free shipping,Not Available,37
8,LG G8 ThinQ - 128GB - Aurora Black (Unlocked) ...,bstco,99.5,Free shipping,Free returns,NaN
9,LG RARE FLIP PHONE Ux280-WINE RED VINTAGE RETR...,theworldofstephjones,100,+$6.05 shipping,Not Available,NaN
